In [9]:
import boto3
import sagemaker
from pyathena import connect
import pandas as pd

## Data Wrangling

In [5]:
# Make athena db
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [6]:
ingest_create_athena_db_passed = False

In [10]:
# Set db name
database_name = "crops"

In [8]:
# Set S3 staging directory - temporary directory used for athena
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [12]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [20]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS crops


""


In [21]:
crops_dir = "s3://sagemaker-studio-998234604495-nyhifbo32oo"

In [25]:
table_name = "crops_long_csv"
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)

create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                Area string,
                Item string,
                Element string,
                Unit string,
                Variable string,
                Value float
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{crops_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count '=1')
"""

pd.read_sql(create_table, conn)
pd.read_sql(f'SELECT * FROM {database_name}.{table_name} LIMIT 5', conn)
                
                


Failed to execute query.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/pyathena/common.py", line 305, in _execute
    **request
  File "/opt/conda/lib/python3.7/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/tenacity/__init__.py", line 379, in __call__
    do = self.iter(retry_state=retry_state)
  File "/opt/conda/lib/python3.7/site-packages/tenacity/__init__.py", line 314, in iter
    return fut.result()
  File "/opt/conda/lib/python3.7/concurrent/futures/_base.py", line 428, in result
    return self.__get_result()
  File "/opt/conda/lib/python3.7/concurrent/futures/_base.py", line 384, in __get_result
    raise self._exception
  File "/opt/conda/lib/python3.7/site-packages/tenacity/__init__.py", line 382, in __call__
    result = fn(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/botocore/client.py", line 530, in _api_call
    return

DatabaseError: Execution failed on sql: 
CREATE EXTERNAL TABLE IF NOT EXISTS crops.crops_long_csv(
                Area string,
                Item string,
                Element string,
                Unit string,
                Variable string,
                Value float
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION 's3://sagemaker-studio-998234604495-nyhifbo32oo/crops_long_csv'
                TBLPROPERTIES ('skip.header.line.count '=1')

An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: line 1:8: mismatched input 'EXTERNAL'. Expecting: 'OR', 'SCHEMA', 'TABLE', 'VIEW'
unable to rollback